In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/home/caleb/Cornell-Conversational-Analysis-Toolkit'

We can use Hypergraph features for various predictive tasks:

In [2]:
from convokit import PairedPrediction

In [3]:
import convokit
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
# os.chdir('..')

In [8]:
reddit_corpus_dir = "/sauna/reddit_201810_raw/reddit-corpus-2/reddit-corpus"

In [9]:
# corpus = convokit.Corpus(filename=convokit.download('reddit-corpus'))
corpus = convokit.Corpus(filename=os.path.join(reddit_corpus_dir, 'reddit-corpus'))

In [17]:
hc = convokit.HyperConvo(prefix_len=10, min_thread_len=10, include_root=False)
# hc.fit_transform(corpus)

In [9]:
remake_cache = True
if remake_cache:
    with open("hyperconvo_feats.p", "wb") as f:
        hyperconvo_feats = {}
        for convo in corpus.iter_conversations():
            hyperconvo_feats.update(convo.meta["hyperconvo"])
        pickle.dump(hyperconvo_feats, f)
else:
    with open("hyperconvo_feats.p", "rb") as f:
        hyperconvo_feats = pickle.load(f)

In [10]:
threads = corpus.utterance_threads(include_root=False)

In [11]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=10, include_root=False)

In [12]:
len(threads)

100000

In [13]:
corpus.print_summary_stats()

Number of Users: 521777
Number of Utterances: 2004262
Number of Conversations: 84979


In [14]:
from collections import defaultdict
thread_roots_by_self_post = defaultdict(list)
for top_level_comment, thread in threads.items():
    rt = thread[next(iter(thread))].root
    thread_roots_by_self_post[rt].append(top_level_comment)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
hyperconvo_feats = hc.retrieve_feats(corpus)

In [19]:
path_stats = hc.retrieve_motif_pathway_stats(corpus)

In [20]:
motif_counts = hc.retrieve_motif_counts(corpus)

In [21]:
threads_motifs = hc.retrieve_motifs(corpus)

In [20]:
# latent_motif_count = {thread_id: hc._latent_motif_count(motif_dict, trans=False)[0] for thread_id, motif_dict in threads_motifs.items()}

In [22]:
hyperconv_df = pd.DataFrame.from_dict(hyperconvo_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [23]:
path_stats_df = pd.DataFrame.from_dict(path_stats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns
path_feat_names = list(path_stats_df.columns)

In [24]:
motif_counts_df = pd.DataFrame.from_dict(motif_counts, orient='index')
motif_feat_names = list(motif_counts_df.columns)

In [24]:
# latentmotif_df = pd.DataFrame.from_dict(latent_motif_count, orient='index')
# latentmotif_df.columns = ['LATENT_'+c for c in latentmotif_df.columns]
# latent_motif_feat_names = list(latentmotif_df.columns)

In [25]:
def get_num_users(thread):
    return len(set(utt.user.name for utt in thread.values()))

thread_to_usercount = dict()
for thread_id in thread_pfxs:
    thread_to_usercount[thread_id] = {"num_users": get_num_users(thread_pfxs[thread_id])}

In [26]:
num_users_df = pd.DataFrame.from_dict(thread_to_usercount, orient='index')
num_users_feat = ['num_users']

In [27]:
feats_df = pd.concat([hyperconv_df, path_stats_df, motif_counts_df, num_users_df], axis=1)

In [28]:
# first generate positive and negative examples based on task
import random
def generate_pos_neg(task: str, post_to_thread_obj, threads, thread_pfxs):
    pos, neg = [], []
    if task == "comment-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos = [root for root in thread_roots if len(threads[root]) >= 15]
            has_neg = [root for root in thread_roots if len(threads[root]) == 10]
            
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    elif task == "commenter-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for root in thread_roots:
                if len(threads[root]) >= 20:
                    if len(set(c.user.name for c in threads[root].values())) >= \
                        len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                            has_pos.append(root)
                    else:
                        has_neg.append(root)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    print("- {} positive, {} negative pts for {} task".format(len(pos), len(neg), task))
    
    return pos, neg


In [29]:
pos_comment_growth, neg_comment_growth = generate_pos_neg("comment-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 1827 positive, 1827 negative pts for comment-growth task


In [30]:
pos_commenter_growth, neg_commenter_growth = generate_pos_neg("commenter-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

- 849 positive, 849 negative pts for commenter-growth task


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import cross_val_score


for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)
    for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (hyperconv_feat_names + num_users_feat, "hyperconv-usercount"),
#                         (latent_motif_feat_names, "latentmotif"),
#                         (latent_motif_feat_names + num_users_feat, "latentmotif-usercount"),
                        (motif_feat_names, "motif"),
                        (motif_feat_names, "motif-usercount"),
                        (path_feat_names, "motifpaths"),
                        (path_feat_names + num_users_feat, "motifpaths-usercount"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
#                         (hyperconv_feat_names + latent_motif_feat_names, "hyperconv-latent"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names, "hyperconvo-motifall"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names + num_users_feat, "hyperconvo-motifall"),
                        (num_users_feat, "usercount")
                       ]:
        clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      
        loo = LeaveOneOut()
        pp = PairedPrediction()
        X, y = pp._generate_paired_X_y(feats_df[feature_set], pos, neg)
        print(X.shape)
        scores = cross_val_score(clf, X, y, cv=loo)
        print("- {}, cv_accuracy: {:.4f}".format(name, scores.mean()))


#         print("Feature set: {}".format(name))
#         pp.fit_predict(feats_df[feature_set], pos, neg, test_size=0.2)
#         pp.print_extreme_coefs(feature_set, num_features=5)

TASK: comment-growth

- 1827 positive, 1827 negative pts for comment-growth task
Excluded 46 data point(s) that contained NaN values.
(1781, 144)
- hyperconvo, cv_accuracy: 0.5969
Excluded 46 data point(s) that contained NaN values.
(1781, 145)
- hyperconv-usercount, cv_accuracy: 0.5957
(1827, 16)
- motif, cv_accuracy: 0.5939
(1827, 100)
- motifpaths, cv_accuracy: 0.5889
(1827, 101)
- motifpaths-usercount, cv_accuracy: 0.6026
Excluded 46 data point(s) that contained NaN values.
(1781, 160)
- hyperconv-motif, cv_accuracy: 0.5957
Excluded 46 data point(s) that contained NaN values.
(1781, 244)
- hyperconv-paths, cv_accuracy: 0.6002
Excluded 46 data point(s) that contained NaN values.
(1781, 260)
- hyperconvo-motifall, cv_accuracy: 0.6002
Excluded 46 data point(s) that contained NaN values.
(1781, 261)


In [51]:
pos, neg = generate_pos_neg("comment-growth", thread_roots_by_self_post, threads, thread_pfxs)

- 1827 positive, 1827 negative pts for comment-growth task


In [52]:
threads_text = dict()
for thread_id in pos + neg:
    threads_text[thread_id] = {"text": " ".join(utt.text for utt in thread_pfxs[thread_id].values())}

In [53]:
text_df = pd.DataFrame.from_dict(threads_text, orient='index')

In [54]:
# First split the pos + neg pairs in train, test sets

In [55]:
from sklearn.model_selection import train_test_split
pos_neg_train, pos_neg_test = train_test_split(list(zip(pos, neg)), test_size=0.2, random_state=42)

In [56]:
pos_train = [x[0] for x in pos_neg_train]
neg_train = [x[1] for x in pos_neg_train] 
pos_test = [x[0] for x in pos_neg_test]
neg_test = [x[1] for x in pos_neg_test]

In [57]:
# BOW vectorizer on train dataset

In [58]:
train_text_df = text_df.loc[pos_train + neg_train]

In [59]:
test_text_df = text_df.loc[pos_test + neg_test]

In [60]:
len(test_text_df)

732

In [61]:
cv = CountVectorizer(min_df=0.15, max_df=0.7)

In [62]:
train_text_arr = cv.fit_transform(train_text_df['text'])

In [63]:
train_text_transform_df = pd.DataFrame(train_text_arr.toarray(), columns=cv.get_feature_names(), index=train_text_df.index)

In [64]:
train_text_transform_df.head()

,about,actually,after,again,all,already,also,always,am,an,...,will,without,work,would,wouldn,wrong,yeah,years,yes,your
dsmhsbk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e3d0msv,0,2,0,0,1,0,1,0,0,6,...,0,0,0,0,0,0,1,2,0,0
dxn8zba,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dz19xqz,1,0,0,0,4,0,1,2,1,2,...,1,0,0,1,0,0,0,0,0,1
e47n813,3,0,0,0,1,0,0,0,0,3,...,5,0,1,2,0,1,0,0,0,12


In [65]:
test_text_arr = cv.transform(test_text_df['text'])
test_text_transform_df = pd.DataFrame(test_text_arr.toarray(), columns=cv.get_feature_names(), index=test_text_df.index)   


In [68]:
from pandas import DataFrame
import numpy as np
def _generate_paired_X_y(feats: DataFrame, pos_ids, neg_ids):

    X, y = [], []
    flip = True

    excluded = 0
    for idx in range(len(pos_ids)):
        pos_feats = np.array(feats.loc[pos_ids[idx]])
        neg_feats = np.array(feats.loc[neg_ids[idx]])

        if (np.isnan(pos_feats).any() or np.isnan(neg_feats).any()):
            excluded += 1
            continue

        if flip:
            y.append(1)
            diff = pos_feats - neg_feats
        else:
            y.append(0)
            diff = neg_feats - pos_feats

        X.append(diff)
        flip = not flip

    if excluded > 0:
        print("Excluded {} data point(s) that contained NaN values.".format(excluded))

    return np.array(X), np.array(y)

In [69]:
X_train, y_train = _generate_paired_X_y(train_text_transform_df, pos_train, neg_train)

In [70]:
X_test, y_test = _generate_paired_X_y(test_text_transform_df, pos_test, neg_test)

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])       

In [72]:
X_train.shape

(1461, 191)

In [73]:
clf = clf.fit(X_train, y_train)

In [74]:
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
print("- {}: {:.4f} train, {:.4f} test".format("cumulative_bow", train_acc, test_acc))

- cumulative_bow: 0.6413 train, 0.4945 test


In [75]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [76]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features=20)


TOP 20 FEATURES
my: 0.179
than: 0.163
also: 0.158
going: 0.157
both: 0.145
all: 0.142
how: 0.139
he: 0.133
we: 0.132
thought: 0.129
like: 0.128
had: 0.124
from: 0.120
any: 0.119
everyone: 0.113
don: 0.105
let: 0.099
them: 0.098
only: 0.096
why: 0.091

BOTTOM 20 FEATURES
him: -0.105
maybe: -0.105
least: -0.110
man: -0.115
same: -0.116
up: -0.117
enough: -0.117
few: -0.118
lot: -0.121
would: -0.132
here: -0.148
oh: -0.149
first: -0.150
who: -0.161
there: -0.188
know: -0.194
got: -0.198
can: -0.214
was: -0.264
re: -0.268

